# Another introduction to FFTs, leading to a discussion of noise

Each cell in this notebook needs to be executed in sequence. We begin by importing the libraries that we will be using.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

%matplotlib inline

We are now going to generate some arrays of samples, and see what happens when we calculate their FFTs.

Incidentally, the first "F" in "FFT" stands for "Fast", and this is because it is calculated using an algorithm published in 1965 by Cooley and Tukey. The algorithm reduces the time required to calculate the Fourier transform by realising that many of the terms are duplicated. It is $O(N\log N)$ rather than $O(N^2)$, so it becomes increasingly valuable for large numbers of samples. The Cooley-Tukey algorithm works best for sample sizes that are a power of two, so it can be an advantage to pad your samples with zeros up to the next power of two. The algorithm works worst for a prime number of samples.

Anyway... back to our explorations...

In [ ]:
# Choose a nice large number of samples, and make it
# a power of 2, which is ideal for the FFT algorithm.

numSamples = 2**10

# Set the time between samples in seconds to 0.001 second. This 
# give us a Nyquist frequency (the highest frequency we can sample
# without aliasing) of 500 Hz.

dt = 0.001

# Now generate an array of times, beginning at zero, separated by dt.

t = np.linspace(0, dt * (numSamples - 1), numSamples)

# In Fourier space, this array of times corresponds to an array of
# frequencies, so lets calculate that.

f = np.fft.fftfreq(t.size, dt)

# Ok, so we are now ready to create a signal. Let's try a sine wave
# at 400 Hz.

freqHz = 400.0
y = np.sin(2 * np.pi * freqHz * t)

# Now find the FFT of the signal, and plot its absolute value versus frequency.

fft = np.fft.fft(y)
plt.plot(f, abs(fft))
plt.xlabel("Frequency [Hz]")
plt.show()

Notice how you get +ve and -ve frequencies. The negative frequencies are only relevant for complex input signals, and since we will almost always be dealing with real input signals (i.e., from measurements of voltages, currents, etc), we can ignore the negative frequencies, and just plot the +ve ones as follows:

In [ ]:
# The +ve frequencies are in the first half of the FFT arrays.
# Note that we use integer division to find the correct array slice endpoint.

plt.plot(f[:numSamples//2], abs(fft[:numSamples//2]), '.')
plt.xlabel("Frequency [Hz]")
plt.show()

But a simpler approach is to use the numpy routines `rfft` and `rfftfreq` that are specifically designed for real samples, and return only the positive frequencies. We also multiply the FFT by $2/n$ where $n$ is the number of samples, so that the amplitude is correct.

In [ ]:
# Find the frequency bins. Note that we are calling rfftfreq, for real samples.

f = np.fft.rfftfreq(t.size, dt)

# Now find the FFT of the signal using rfft, normalise by multiplying by
# 2/numSamples, and plot the absolute value versus frequency.

fft = np.fft.rfft(y) * 2 / numSamples
plt.plot(f, abs(fft), 'o')
plt.xlabel("Frequency [Hz]")
plt.show()

You may wonder why there is a *distribution* of frequencies in the FFT, whereas we created the signal from a pure sine wave. The reason is that the FFT assumes that the input signal is *periodic*, so even though we only provided 1024 samples, the FFT assumes that the signal continues forever, and is constructed by taking copies of the 1024 samples and adding them to the end of our original 1024, and so on. Unless we choose the input frequency very carefully, there will be a discontinuity at the join, and the FFT will find a range of sine waves that do good job of trying to follow the discontinuity.

The spilling of power into neighbouring frequencies is an example of *spectral leakage*. There is not much you can do about it when using a FFT. Sampling the input waveform more frequently doesn't help. What does help is to increase the observation time, and in the limit that you sample for infinity seconds, you get a perfect result!

To explore this further, let's choose a frequency near 400 Hz that gives an exact integer number of sine waves in 1024 samples at 0.001 second per sample.

In [ ]:
# Choose a frequency to fit an integral number of cycles in 1024 samples.

freqHz = 410 / (1024 * 0.001)
print("the frequency is", freqHz, "Hz")

# Create the signal.

y = np.sin(2 * np.pi * freqHz * t)
f = np.fft.rfftfreq(t.size, dt)

# Calculate the FFT and plot it.

fft = np.fft.rfft(y) * 2 / numSamples

plt.plot(f, abs(fft))
plt.xlabel("Frequency [Hz]")
plt.show()

# Plot it again, this time zooming in on the relevant section.

plt.plot(f, abs(fft), 'o')
plt.xlim(390, 410)
plt.xlabel("Frequency [Hz]")
plt.show()


You can see that all the power is now at a single frequency, since the infinite repetition of our samples has no discontinuities.

# Power spectrum

For many purposes it is more useful to plot the *power spectrum* rather than the amplitude spectrum (i.e., the absolute value of the FFT). Since power goes as the square of the amplitude, the power spectrum simply comes from squaring the absolute value of the FFT, or equivalently, multiplying the FFT by its complex conjugate.

In [ ]:
# Plot the power spectrum, i.e., the square of the absolute value of the FFT.

plt.plot(f, abs(fft) ** 2)
plt.ylabel("Power")
plt.xlabel("Frequency [Hz]")
plt.show()

# We get the same result by multiplying the FFT by its complex
# conjugate. We take the real part to stop plot() from complaining -
# the imaginary part should be all zeros anyway.

plt.plot(f, np.real(fft * np.conj(fft)))
plt.ylabel("Power")
plt.xlabel("Frequency [Hz]")
plt.show()

# What do the real and imaginary parts of the FFT mean?

So far we have been plotting the absolute value of the FFT. The absolute value tells us the how much of a particular frequency is in the input array. It tells us nothing about the phase relationship between the frequencies; very different waveforms can be produced by shifting the phases of the frequencies. For example, the following snippet generates four input waveforms that have identical absolute values of their FFTs.

In [ ]:
freqHz = 5 / (1024 * 0.001)
print("the frequency is", freqHz, "Hz")

# Create four signals, with the same frequencies and amplitudes, 
# but different phases. We displace them in the y direction to
# make them easy to see.

y0 = np.sin(2 * np.pi * freqHz * t) + 0.3 * np.sin(2 * np.pi * (freqHz + freqHz) * t)
y1 = np.cos(2 * np.pi * freqHz * t) + 0.3 * np.sin(2 * np.pi * (freqHz + freqHz) * t)
y2 = np.sin(2 * np.pi * freqHz * t) + 0.3 * np.cos(2 * np.pi * (freqHz + freqHz) * t)
y3 = np.cos(2 * np.pi * freqHz * t) + 0.3 * np.cos(2 * np.pi * (freqHz + freqHz) * t)

plt.plot(y0 - 3.0)
plt.plot(y1 - 1.0)
plt.plot(y2 + 1.0)
plt.plot(y3 + 3.0)
plt.xlabel("Time [ms]")
plt.show()


Now lets plot the absolute value, argument, real, and imaginary parts of the FFT for each of these signals. I have only shown y0 in this notebook, you need to edit this to explore y1, y2, and y3.

You need to look at the plots very carefully. The argument plot looks chaotic, but that is simply because the real and imaginary components are at the level of floating-ppint noise apart from at the two input frequencies (4.88 Hz and 2 * 4.88 Hz) - so only look at the values at these two points. Also, keep an eye on the scaling factor to be applied to the y-axis.

With some careful study you should see the following:

- the absolute value plot is the same for all four signals
- the argument plot shows the phase relationship of the component frequencies
- the real plot shows the cosine component
- the imaginary plot shows minus the sine component

The important thing to take away from this is that there is nothing mysterious about the complex nature of the FFT. It is simply that you need two numbers (amplitude and phase) to represent a frequency component in a signal, and the easiest way to do this is using complex numbers. In particular, there is nothing strange of mysterious about the *imaginary* component of an FFT.

In [ ]:
def plotFFT(y):

    # Calculate the FFT, normalise it, then plot.
    
    fft = np.fft.rfft(y) * 2 / numSamples

    plt.plot(f, abs(fft), 'o')
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Absolute value of FFT")
    plt.xlim(0,20)
    plt.show()

# Plot it again, this time zooming in on the relevant section.

    plt.plot(f, np.angle(fft), 'o')
    plt.xlim(0, 20)
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Arg of FFT")
    plt.show()

    plt.plot(f, np.real(fft), 'o')
    plt.xlim(0, 20)
    plt.ylabel("Real value of FFT")
    plt.xlabel("Frequency [Hz]")
    plt.show()

    plt.plot(f, np.imag(fft), 'o')
    plt.xlim(0, 20)
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Imaginary value of FFT")
    plt.show()

# Now actually plot a signal! You should edit y0 to be y1, y2, and then y3, and
# examine the differences.

plotFFT(y0)

# Noise

The above discussion has concentrated on examining clear sinusoidal signals, but what happens if the samples are noisy?

## White noise

Let's begin with two sine-waves (chosen to have an integral number of cycles in our sample window) with some Gaussian noise added.

In [ ]:
# Choose a couple of frequencies, one low, one high.

freq1Hz = 5 / (1024 * 0.001)
print("the 1st frequency is", freq1Hz, "Hz")

freq2Hz = 410 / (1024 * 0.001)
print("the 2nd frequency is", freq2Hz, "Hz")

# Create the samples.

y = np.sin(2 * np.pi * freq1Hz * t) + np.sin(2 * np.pi * freq2Hz * t) + 2.0 * np.random.randn(numSamples) 

# And plot them.

plt.plot(t, y)
plt.show()

# Calculate the FFT, and plot its power spectrum.

fft = np.fft.rfft(y) * 2 / numSamples
plt.plot(f, abs(fft) ** 2, '.')
plt.xlabel("Frequency [Hz]")
plt.ylabel("Power")
plt.show()


Note how the two signal frequencies are clearly detected (although their power isn't exactly 1.0), and the noise appears with roughly equal amplitude at all frequencies.

If we average the amplitude spectrum from many random samples, the noise frequency spectrum becomes clearer. Try the following code snippet with various values of $n$. Note the scaling on the y axis.

In [ ]:
def plotAverageFFT(n):
    numSamples = 1024
    dt = 0.001
    t = np.linspace(0, dt * (numSamples - 1), numSamples)
    f = np.fft.rfftfreq(t.size, dt)
    
    # We are going to calculate an average power spectrum,
    # so we begin by zeroing a suitable array.
    
    powerSpectrum = np.zeros(f.size)

    # Now sum n power spectra. 
    
    for i in range(n):
        
        # Uncomment this line for Gaussian distributed samples.
        
        y = np.random.randn(numSamples)
        
        # Uncomment this line for uniform distrubuted samples.
        
        #y = np.random.random(numSamples) - 0.5
        
        powerSpectrum += abs(np.fft.rfft(y) * 2 / numSamples) ** 2
        
    # And divide by n to make the avergage.
    
    powerSpectrum /= n

    plt.plot(f, powerSpectrum)
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Power")
    plt.show()

plotAverageFFT(10)

You can see from the above plots that the noise we added has a constant power for all frequencies, where the power is measured in bins of constant width (in Hz). This is called *white noise*. Note that the fact that we used a Gaussian distribution of sample values at each point isn't important - all that matters for noise to be white is that there is *no correlation* from one sample to the next.

With white noise there is as much noise in the bin from 1-2 Hz as there is from 1,000,000-1,000,001 Hz.

## 1/f noise (pink noise)

In many physical systems, the noise isn't white, but increases to lower frequencies as $1/f$. This is called $1/f$ noise, or sometimes *pink noise*. With $1/f$ noise there is equal power in logarithmic space, e.g., the noise power from 1-10 Hz is the same as from 1,000-10,000 Hz.

Generating pink noise in a computer is a little trickier than white noise. One algorithm for doing it, approximately, is the Voss-McCartney algorithm. I won't explain how this works, but let's try it and see what happens.

In [ ]:
import pandas as pd
def voss(nrows, ncols=16):
    
    # From Allen Downey, Think DSP
    
    """Generates pink noise using the Voss-McCartney algorithm.
    
    nrows: number of values to generate
    rcols: number of random sources to add
    
    returns: NumPy array
    """
    array = np.empty((nrows, ncols))
    array.fill(np.nan)
    array[0, :] = np.random.random(ncols)
    array[:, 0] = np.random.random(nrows)
    
    # the total number of changes is nrows
    n = nrows
    cols = np.random.geometric(0.5, n)
    cols[cols >= ncols] = 0
    rows = np.random.randint(nrows, size=n)
    array[rows, cols] = np.random.random(n)

    df = pd.DataFrame(array)
    df.fillna(method='ffill', axis=0, inplace=True)
    total = df.sum(axis=1)

    return total.values

def plotAverageFFT(n):
    numSamples = 1024
    dt = 0.001
    t = np.linspace(0, dt * (numSamples - 1), numSamples)
    f = np.fft.rfftfreq(t.size, dt)
    
    # As before, let's calculate the average power spectrum for a number
    # of samples.
    
    powerSpectrum = np.zeros(f.size)

    for i in range(n):
        y = voss(numSamples) 
        powerSpectrum += abs(np.fft.rfft(y) * 2 / numSamples) ** 2

    powerSpectrum /= n
    
    # Plot with linear axes.
    
    plt.plot(f, powerSpectrum)
    plt.ylim(0.0,0.1)
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Power")
    plt.show()
    
    # A log-log plot shows the noise spectrum more clearly.
    
    plt.plot(f, powerSpectrum)
    plt.loglog()
    plt.ylim(0.0001,1)
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Power")
    plt.show()

plotAverageFFT(1)

You can see how the noise follows a $1/f$ power spectrum - the slope is $-1$ in the log-log plot. Also note that the slope isn't perfect. I don't know if this is a limitation of the Voss-McCartney algorithm, or the finite length of time over which the noise is sampled.

## $1/f^2$ red noise

If the power spectrum slope goes as $1/f^2$ the noise is referred to as red noise, or sometimes brown noise (the "brown" comes from "Brownian motion", which generates this type of noise).

# Physical reasons for noise

## Shot noise (also known as photon noise, Poisson noise, or Schottky noise)

If the phenomenon under observation is inherently quantised (e.g., radioactive decay, the arrival of photons, the current due to electrons, rain drops on a roof), then Poissonion statistics gives the distribution of samples. 

The standard deviation of the samples in a Poisson distribution is given by $\sqrt N$ where $N$ is the number of samples. The signal-to-noise ratio is therefore ${\rm SNR}=N/\sqrt N=\sqrt N$, from which it is clear that shot noise only tents to be important when $N$ is small, so that the individual photons/electroncs/etc can be readily detected. At large values of $N$, other sources of noise will dominate.

Shot noise is an example of white noise, i.e., it has a constant power per Hz at all frequencies.

If you consider the flow of electron current in a wire, the mean square of the current noise is given by

$$\langle I^2_\hbox{noise}\rangle=2q\langle I\rangle\Delta f$$

where $q$ is the charge of an electron, $I$ is the current, and $\Delta f$ is the bandwidth of the measurement in Hz.


## Johnson noise (also known as thermal noise, or Nyquist noise)

Even if no net current is flowing in a wire, there will still be electrons randomly moving backwards and forwards along the wire as a result of their thermal velocity. This produces a current noise known as Johnson noise. It is normally expressed as the voltage developed across the wire's resistance:

$$\langle V^2_\hbox{noise}\rangle=4kTR\Delta f$$

where $k$ is Boltzman's constant, $T$ is the temperature of the resistor, $R$ its resistance, and $\Delta f$ is again the bandwidth of the measurement in Hz. Note that Johnson noise is independent of the current flow - the same amount of Johnson noise is superimposed on whatever DC current is flowing. Johnson noise provides a lower limit on the noise from a resistor; as the current increases, shot noise (and $1/f$ noise, see below) will kick in.

Like shot noise, Johnson noise is an example of white noise.

## $1/f$ noise (also known as flicker noise, or switching noise)

Many physical systems exhibit $1/f$ noise, and the reasons are many and varied, and not always obvious.

In the case of current flow through conductors, $1/f$ noise appears to result from fluctuations in *conductivity* due to thermal activation/deactivation of defects in the conductor.

In a perfect world a conductor would be perfect metal lattice, and electrons would flow smoothly. In reality, the lattices contain *defects* such as missing atoms, dopant atoms, grain boundaries, and so on. These defects act to scatter electrons, and hence reduce the conductivity. Furthermore, the defects can be in various states depending on temperature, with various relaxation times to lower energy states, and the states have different scattering cross sections for electrons. The end result is that the conductivity fluctuates in a random manner, and with a temperature dependence. Also, as the average DC current increases, the $1/f$ voltage noise increases since a given delta in conductivity will cause a proportionally greater change in voltage.

At low temperatures, the higher energy states are not accessible, and you can be in the regime where the jumps in conductivity are easily visible. The same thing can happen if the current is flowing through a very narrow cross-section, such that a few defects can dominate the conductivity.

Unlike Johnson noise, $1/f$ noise is highly dependent on the physical type of resistor or component under consideration. Annealing a metal film tends to reduce the number of defects, so $1/f$ noise is reduced. Conversely, radiation damage can create defects that lead to an increase in $1/f$ noise.

$1/f$ noise occurs in something as simple as a switch contact (hence the term *switching noise*) as a result of the small cross-section where current has to flow. To reduce this effect, switches can be made with multiple contacts to increase the area.

## Interesting sources of noise

Physics experiments quite often have to work close to the limit of what is possible. As such, minimising noise is something that physicists are often obsessed by. Over the years we tend to accummulate "war stories" describing particularly tricky noise problems that we have overcome. Here are some examples:

- semiconductors are sensitive to light, and so they are usually encapsulated in opaque plastic or ceramic packages. If light can somehow still reach the semiconductor, you may see soem unexpected effects, such as 100 Hz oscillation from fluorescent lamps.

- mechanical vibration can cause induced currents resulting from electric fields and changing capacitances. This is sometimes referred to as "microphonics" since the circuit can pick up sounds waves somewhat like a microphone.

- the tribolectric effect generates current when two dissimilar objects move relative to each other. This movement can be as simple as the bending of a cable consisting of a conductor and an insulator. As such, the triboelectric effect is another way for mechanical vibration to cause voltage noise.

- junctions between dissimilar metals will generate a voltage through the thermoelectric effect. If the temperature is fluctuating, you will see an apparent voltage noise. While this is usually at a fairly low frequency (due to longish thermal time constants) it can be short if the volumes of metals are small. 

- see [this article by myself and Nic Bongham](http://mcba11.phys.unsw.edu.au/~plato-a/papers/bin14a.pdf) for how an interesting noise problem was solved.


